In [1]:
import gymnasium as gym

In [2]:
env = gym.make("cpr_gym_rs:Nakamoto-v0")

In [3]:
def policy(o):
    assert o.shape == (3,)
    a = o[0]
    h = o[1]
    if h > a:
        return [0.0]  # consider / adopt
    if a == 1 and h == 1:
        return [-0.5]  # release / match
    if h == a - 1 and h >= 1:
        return [-0.5]  # release / override
    return [0.0]  # continure / wait

In [4]:
ep_rew = []
ep_len = []
ep_prg = []

acc_rew = 0.0
acc_len = 0
acc_prg = 0
obs, info = env.reset()
for i in range(10000):
    a = policy(obs)
    obs, r, term, trunc, info = env.step(a)
    acc_rew += r
    acc_len += 1
    acc_prg += info["progress"]
    if term or trunc:
        ep_rew.append(acc_rew)
        ep_len.append(acc_len)
        ep_prg.append(acc_prg)
        acc_rew = 0.0
        acc_len = 0
        acc_prg = 0
        obs, info = env.reset()

In [5]:
print(ep_rew)
print(ep_len)
print(ep_prg)

[0.0, 0.0, 6.0, 98.0, 0.0, 1.0, 9.0, 24.0, 10.0, 4.0]
[1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
[0.0, 0.0, 6.0, 98.0, 0.0, 1.0, 9.0, 24.0, 10.0, 4.0]
